In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open("names.txt").read().splitlines()
print(words[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [3]:
len(words)

32033

In [4]:
chs = sorted(list(set("".join(words))))
stoi = {s: i + 1 for i, s in enumerate(chs)}
stoi["."] = 0
itos = {i: s for s, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# building the dataset
block_size = 3
X, Y = [], []

for w in words:
    # print(w)
    ctx = [stoi["."]] * block_size
    for c in w + ".":
        X.append(ctx)
        Y.append(stoi[c])
        # print("".join(itos[i] for i in ctx), "-->", c)
        ctx = ctx[1:] + [stoi[c]]

X, Y = torch.tensor(X), torch.tensor(Y)

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [8]:
sum(p.nelement() for p in parameters) # total no parameters

3481

In [9]:
for p in parameters:
    p.requires_grad = True

In [20]:
batch_size = 32
for _ in range(10000):
    # construct mini_batch
    i = torch.randint(0, X.shape[0], (32,))
    
    # forward pass
    emb = C[X[i]]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y[i])

    # backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    # optimizer step
    for p in parameters:
        p.data -= 0.01 * p.grad

print(loss.item())

1.8938106298446655


In [21]:
emb = C[X]
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Y)
loss

tensor(2.3086, grad_fn=<NllLossBackward0>)